<a href="https://colab.research.google.com/github/avner8943/bootcamp/blob/master/advanced_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Q1.a change number of filters

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        

        self.conv1 = nn.Conv2d(3, 6, 5)

        self.conv2 = nn.Conv2d(6, 16, 5)

        # randomly select number of feagtures
        fet1 = np.random.randint(100, 400)

        fet2 = np.random.randint(100, 400)

        
        self.fc1 = nn.Linear(in_features= 16 * 5 * 5, out_features=fet1)  # 6*6 from image dimension
        
        self.fc2 = nn.Linear(in_features=fet1, out_features=fet2)

        self.fc3 = nn.Linear(in_features=fet2, out_features=n_classes)
    
    def forward(self, x):

        x = F.max_pool2d(torch.relu(self.conv1(x)), kernel_size=2)
        x = F.max_pool2d(torch.relu(self.conv2(x)), kernel_size=2)
  
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        x = self.fc3(x)

        return x




In [2]:
import torch
import torchvision
import torchvision.transforms as transforms


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
import torch.optim as optim

net = LeNet5(len(classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [4]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print(outputs.shape, inputs.shape, labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0


[1,  2000] loss: 2.198
[1,  4000] loss: 1.866
[1,  6000] loss: 1.664
[1,  8000] loss: 1.538
[1, 10000] loss: 1.487
[1, 12000] loss: 1.423
[2,  2000] loss: 1.367
[2,  4000] loss: 1.337
[2,  6000] loss: 1.304
[2,  8000] loss: 1.280
[2, 10000] loss: 1.256
[2, 12000] loss: 1.258


In [5]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 55 %


I run the net few times and always got between 53-57% of success so i guess the number of features is not very meanninful

### Q1.b change conv2d 

In [6]:
class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        # assuming that nmnist image is 28*28

        stride = 2

        k=5



        self.conv1 = nn.Conv2d(3, 6, k,padding=(2,2), stride = 2)

        self.conv2 = nn.Conv2d(6, 16, k,padding=(2, 2), stride = 2)


        
        self.fc1 = nn.Linear(in_features= 64, out_features=120)  
        
        self.fc2 = nn.Linear(in_features=120, out_features=84)

        self.fc3 = nn.Linear(in_features=84, out_features=n_classes)
    
    def forward(self, x):

        x = F.max_pool2d(torch.relu(self.conv1(x)), kernel_size=2)
        x = F.max_pool2d(torch.relu(self.conv2(x)), kernel_size=2)
  
        x = x.view(4, -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        x = self.fc3(x)

        return x




In [7]:
net = LeNet5(len(classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [8]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

[1,  2000] loss: 2.284
[1,  4000] loss: 2.047
[1,  6000] loss: 1.834
[1,  8000] loss: 1.688
[1, 10000] loss: 1.629
[1, 12000] loss: 1.575
[2,  2000] loss: 1.518
[2,  4000] loss: 1.499
[2,  6000] loss: 1.471
[2,  8000] loss: 1.466
[2, 10000] loss: 1.438
[2, 12000] loss: 1.436


In [9]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 49 %


this section the results are very bad, only 50%

### Q1.c change stride to 1 

In [10]:
class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        # assuming that nmnist image is 28*28

        stride = 2

        k=5



        self.conv1 = nn.Conv2d(3, 6, k,padding=(2,2), stride = 1)

        self.conv2 = nn.Conv2d(6, 16, k,padding=(2, 2), stride = 1)


        
        self.fc1 = nn.Linear(in_features= 1024, out_features=120)  
        
        self.fc2 = nn.Linear(in_features=120, out_features=84)

        self.fc3 = nn.Linear(in_features=84, out_features=n_classes)
    
    def forward(self, x):

        x = F.max_pool2d(torch.relu(self.conv1(x)), kernel_size=2)
        x = F.max_pool2d(torch.relu(self.conv2(x)), kernel_size=2)
  
        x = x.view(4, -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        x = self.fc3(x)

        return x

In [11]:
net = LeNet5(len(classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [31]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 10):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data


        break
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

In [13]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 9 %


little better then before

### Q2 saving and loading network

**first method**

In [40]:
def save_checkpoint(iters, epoch, net, is_best = 0):
  
  f_name = "net_wights_"+ str(epoch) + "_" + str(iters)

  torch.save(net.state_dict(), f_name)

  f_name_best = "best_net"

  if is_best: 
      torch.save(net.state_dict(), f_name_best)


def load_checkpoint(f_name):

 

  m_state_dict = torch.load(f_name)
  new_net = LeNet5(len(classes))
  new_net.load_state_dict(m_state_dict)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

  return new_net, criterion, optimizer


def eval_net(net):

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct

In [29]:
def train_loop(epoch_l , iters_l, net):

  is_best = 0
  for epoch in range(epoch_l, 2):  # loop over the dataset multiple times

 

      
      running_loss = 0.0
      for i, data in enumerate(trainloader, 0):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data

          if epoch <= epoch_l and i <= iters_l: continue # make sure trainning won't start before previous checkpoint

          #print(epoch)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = net(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          running_loss += loss.item()
          if i % 2000 == 1999:    # print every 2000 mini-batches

              print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
              score = eval_net(net)
              if  score > is_best:
                save_checkpoint(i, epoch, net, 1)
                is_best = score

              running_loss = 0.0
  return net

In [28]:
def load_and_train(f_name = ''):

  if f_name == '':
    new_net = LeNet5(len(classes))
    net = train_loop(0, -1,new_net)
    return net

  
  new_net, criterion, optimizer = load_checkpoint(f_name)

  s = f_name.split("_")

  epoch = s[2]

  iters = s[3]


  net = train_loop(int(epoch), int(iters),new_net)

  return net
  

In [ ]:
net = load_and_train()

In [ ]:
# example of train the network from checkpoint

net = load_and_train('net_wights_1_1999')

**Second method**

In [41]:
def save_checkpoint2(epoch, net, optimizer, is_best = 0):

  f_name = "net_wights_epoch_"+ str(epoch) +".pt"


  state = {
    'epoch': epoch,
    'state_dict': net.state_dict(),
    'optimizer': optimizer.state_dict()}

  torch.save(state, f_name)

  f_name_best = "best_net"

  if is_best: 
      torch.save(net.state_dict(), f_name_best)


def load_checkpoint2(f_name, net, optimizer):

  state = torch.load(f_name)

  new_net = LeNet5(len(classes))

  new_net.load_state_dict(state['state_dict'])
  optimizer.load_state_dict(state['optimizer'])

  return new_net, optimizer, state['epoch']



In [42]:
def train_loop2(epoch_l , net, optimizer):


  for epoch in range(epoch_l, 2):  # loop over the dataset multiple times

      
      running_loss = 0.0
      for i, data in enumerate(trainloader, 0):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data

          #print(epoch)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = net(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          running_loss += loss.item()
          if i % 2000 == 1999:    # print every 2000 mini-batches
             
              print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
              score = eval_net(net)
              if  score > is_best:
                save_checkpoint2(epoch, net, optimizer, 1)
                is_best = score
              running_loss = 0.0

  return net

In [ ]:
net = LeNet5(len(classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

net = train_loop2(0, net, optimizer)

In [ ]:
net, optimizer, start_epoch = load_checkpoint2('net_wights_epoch_1.pt', net, optimizer)

train_loop2(start_epoch, net, optimizer)